In [9]:
import os
import urllib.request
import mxnet as mx

# Download pre-trained Resnet-18 model to directory
# For downloading a file to local disk
def download(url,target_dir):
    filename = url.split('/')[-1]
    if not os.path.exists(os.path.join(target_dir,filename)):
        urllib.request.urlretrieve(url,os.path.join(target_dir,filename))
        
download('http://data.dmlc.ml/mxnet/models/imagenet/resnet/18-layers/resnet-18-0000.params','./model')
download('http://data.dmlc.ml/mxnet/models/imagenet/resnet/18-layers/resnet-18-symbol.json','./model')        

1.1.0


In [2]:
import mxnet as mx

# Load the pre-trained model
sym, arg_params, aux_params = mx.model.load_checkpoint('./model/resnet-18', 0)

# Get Data Iterators  image data and labels.
def get_iterators(batch_size, data_shape=(3, 224, 224)):
    train = mx.io.ImageRecordIter(
        path_imgrec         = './data/cars_train.rec',
        data_name           = 'data',
        label_name          = 'softmax_label',
        batch_size          = batch_size,
        data_shape          = data_shape,
        shuffle             = True,
        rand_crop           = True,
        rand_mirror         = True)
    val = mx.io.ImageRecordIter(
        path_imgrec         = './data/cars_val.rec',
        data_name           = 'data',
        label_name          = 'softmax_label',
        batch_size          = batch_size,
        data_shape          = data_shape,
        rand_crop           = False,
        rand_mirror         = False)
    return (train, val)

In [3]:
# Remove the ImageNet 1000 layer classification layer and attach 196 dimensional cars layer
def get_fine_tune_model(symbol, arg_params, num_classes, layer_name='flatten0'):
    """
    symbol: the pretrained network symbol
    arg_params: the argument parameters of the pretrained model
    num_classes: the number of classes for the fine-tune datasets
    layer_name: the layer name before the last fully-connected layer
    """
    all_layers = symbol.get_internals()
    net = all_layers[layer_name+'_output'] # remove the original fully connected layer
    net = mx.symbol.FullyConnected(data=net, num_hidden=num_classes, name='fc1')
    net = mx.symbol.SoftmaxOutput(data=net, name='softmax')
    mx.viz.plot_network(net) # Visualize the network
    new_args = dict({k:arg_params[k] for k in arg_params if 'fc1' not in k})
    return (net, new_args)

In [ ]:
# Train the model
import logging
head = '%(asctime)-15s %(message)s'
logging.basicConfig(level=logging.DEBUG, format=head)

def fit(symbol, arg_params, aux_params, train, val, batch_size, num_gpus, num_epoch):
    devs = [mx.gpu(i) for i in range(num_gpus)]
    mod = mx.mod.Module(symbol=symbol, context=devs)
    mod.fit(train, val,
        num_epoch=num_epoch, # Number of epochs
        arg_params=arg_params,
        aux_params=aux_params,
        allow_missing=True,
        batch_end_callback = mx.callback.Speedometer(batch_size, 10),
        kvstore='device',
        optimizer='sgd',
            #adam or sgd
        optimizer_params={'learning_rate':0.02}, # Learning rate
        initializer=mx.init.Xavier(rnd_type='gaussian', factor_type="in", magnitude=2),
        eval_metric='acc',
        epoch_end_callback = mx.callback.do_checkpoint('./model/cars-finetuned'))
    metric = mx.metric.Accuracy()
    return mod.score(val, metric)

num_classes = 196 # Number of categories in cars dataset
batch_per_gpu = 200 # Batch size
num_gpus = 1 # Since we only have a single GPU
num_epoch = 500 # Number of epochs that we are going to train for

(new_sym, new_args) = get_fine_tune_model(sym, arg_params, num_classes)

batch_size = batch_per_gpu * num_gpus
(train, val) = get_iterators(batch_size)
mod_score = fit(new_sym, new_args, aux_params, train, val, batch_size, num_gpus, num_epoch)

2018-04-17 22:06:03,854 Epoch[0] Batch [10]	Speed: 608.27 samples/sec	accuracy=0.948636
2018-04-17 22:06:06,128 Epoch[0] Batch [20]	Speed: 880.37 samples/sec	accuracy=0.974500
2018-04-17 22:06:08,213 Epoch[0] Batch [30]	Speed: 959.91 samples/sec	accuracy=0.972000
2018-04-17 22:06:10,295 Epoch[0] Batch [40]	Speed: 960.89 samples/sec	accuracy=0.972000
2018-04-17 22:06:10,296 Epoch[0] Train-accuracy=nan
2018-04-17 22:06:10,296 Epoch[0] Time cost=9.842
2018-04-17 22:06:10,553 Saved checkpoint to "./model/cars-finetuned-0001.params"
2018-04-17 22:06:15,433 Epoch[0] Validation-accuracy=0.585854
2018-04-17 22:06:17,650 Epoch[1] Batch [10]	Speed: 1013.14 samples/sec	accuracy=0.995909
2018-04-17 22:06:19,764 Epoch[1] Batch [20]	Speed: 946.68 samples/sec	accuracy=0.998500
2018-04-17 22:06:21,856 Epoch[1] Batch [30]	Speed: 956.56 samples/sec	accuracy=0.998500
2018-04-17 22:06:23,935 Epoch[1] Batch [40]	Speed: 962.72 samples/sec	accuracy=0.997500
2018-04-17 22:06:23,935 Epoch[1] Train-accuracy=nan

2018-04-17 22:09:16,791 Epoch[14] Batch [20]	Speed: 942.55 samples/sec	accuracy=0.999500
2018-04-17 22:09:18,888 Epoch[14] Batch [30]	Speed: 954.28 samples/sec	accuracy=0.999000
2018-04-17 22:09:20,768 Epoch[14] Train-accuracy=0.999444
2018-04-17 22:09:20,769 Epoch[14] Time cost=8.484
2018-04-17 22:09:21,037 Saved checkpoint to "./model/cars-finetuned-0015.params"
2018-04-17 22:09:25,645 Epoch[14] Validation-accuracy=0.650122
2018-04-17 22:09:27,863 Epoch[15] Batch [10]	Speed: 1014.10 samples/sec	accuracy=0.998182
2018-04-17 22:09:29,968 Epoch[15] Batch [20]	Speed: 951.61 samples/sec	accuracy=0.999500
2018-04-17 22:09:32,068 Epoch[15] Batch [30]	Speed: 953.13 samples/sec	accuracy=0.999500
2018-04-17 22:09:34,160 Epoch[15] Batch [40]	Speed: 956.43 samples/sec	accuracy=0.999000
2018-04-17 22:09:34,161 Epoch[15] Train-accuracy=nan
2018-04-17 22:09:34,161 Epoch[15] Time cost=8.515
2018-04-17 22:09:34,423 Saved checkpoint to "./model/cars-finetuned-0016.params"
2018-04-17 22:09:39,068 Epoch

2018-04-17 22:12:28,029 Epoch[28] Batch [30]	Speed: 953.36 samples/sec	accuracy=0.999000
2018-04-17 22:12:29,913 Epoch[28] Train-accuracy=0.999444
2018-04-17 22:12:29,914 Epoch[28] Time cost=8.508
2018-04-17 22:12:30,174 Saved checkpoint to "./model/cars-finetuned-0029.params"
2018-04-17 22:12:34,692 Epoch[28] Validation-accuracy=0.659500
2018-04-17 22:12:36,910 Epoch[29] Batch [10]	Speed: 1011.75 samples/sec	accuracy=0.999091
2018-04-17 22:12:39,026 Epoch[29] Batch [20]	Speed: 945.57 samples/sec	accuracy=0.998500
2018-04-17 22:12:41,127 Epoch[29] Batch [30]	Speed: 952.62 samples/sec	accuracy=0.998500
2018-04-17 22:12:43,218 Epoch[29] Batch [40]	Speed: 956.76 samples/sec	accuracy=0.999500
2018-04-17 22:12:43,219 Epoch[29] Train-accuracy=nan
2018-04-17 22:12:43,220 Epoch[29] Time cost=8.527
2018-04-17 22:12:43,482 Saved checkpoint to "./model/cars-finetuned-0030.params"
2018-04-17 22:12:48,118 Epoch[29] Validation-accuracy=0.657805
2018-04-17 22:12:50,504 Epoch[30] Batch [10]	Speed: 951

2018-04-17 22:15:41,896 Epoch[42] Train-accuracy=0.998889
2018-04-17 22:15:41,897 Epoch[42] Time cost=8.487
2018-04-17 22:15:42,155 Saved checkpoint to "./model/cars-finetuned-0043.params"
2018-04-17 22:15:47,133 Epoch[42] Validation-accuracy=0.664250
2018-04-17 22:15:49,355 Epoch[43] Batch [10]	Speed: 1015.86 samples/sec	accuracy=0.997727
2018-04-17 22:15:51,466 Epoch[43] Batch [20]	Speed: 948.04 samples/sec	accuracy=0.999500
2018-04-17 22:15:53,579 Epoch[43] Batch [30]	Speed: 947.22 samples/sec	accuracy=0.998500
2018-04-17 22:15:55,667 Epoch[43] Batch [40]	Speed: 958.37 samples/sec	accuracy=0.999000
2018-04-17 22:15:55,668 Epoch[43] Train-accuracy=nan
2018-04-17 22:15:55,668 Epoch[43] Time cost=8.534
2018-04-17 22:15:55,930 Saved checkpoint to "./model/cars-finetuned-0044.params"
2018-04-17 22:16:00,641 Epoch[43] Validation-accuracy=0.664878
2018-04-17 22:16:03,028 Epoch[44] Batch [10]	Speed: 951.39 samples/sec	accuracy=0.998636
2018-04-17 22:16:05,130 Epoch[44] Batch [20]	Speed: 952

2018-04-17 22:18:53,779 Epoch[56] Train-accuracy=nan
2018-04-17 22:18:53,780 Epoch[56] Time cost=8.724
2018-04-17 22:18:54,147 Saved checkpoint to "./model/cars-finetuned-0057.params"
2018-04-17 22:18:58,903 Epoch[56] Validation-accuracy=0.666625
2018-04-17 22:19:01,277 Epoch[57] Batch [10]	Speed: 994.47 samples/sec	accuracy=0.998182
2018-04-17 22:19:03,391 Epoch[57] Batch [20]	Speed: 946.72 samples/sec	accuracy=0.999500
2018-04-17 22:19:05,501 Epoch[57] Batch [30]	Speed: 948.38 samples/sec	accuracy=0.999000
2018-04-17 22:19:07,380 Epoch[57] Train-accuracy=0.999444
2018-04-17 22:19:07,381 Epoch[57] Time cost=8.477
2018-04-17 22:19:07,638 Saved checkpoint to "./model/cars-finetuned-0058.params"
2018-04-17 22:19:12,299 Epoch[57] Validation-accuracy=0.665375
2018-04-17 22:19:14,483 Epoch[58] Batch [10]	Speed: 1016.06 samples/sec	accuracy=0.997727
2018-04-17 22:19:16,581 Epoch[58] Batch [20]	Speed: 953.77 samples/sec	accuracy=0.999500
2018-04-17 22:19:18,691 Epoch[58] Batch [30]	Speed: 949

...

2018-04-17 22:47:34,544 Epoch[183] Batch [20]	Speed: 953.82 samples/sec	accuracy=1.000000
2018-04-17 22:47:36,638 Epoch[183] Batch [30]	Speed: 955.75 samples/sec	accuracy=0.998500
2018-04-17 22:47:38,726 Epoch[183] Batch [40]	Speed: 958.10 samples/sec	accuracy=0.999500
2018-04-17 22:47:38,727 Epoch[183] Train-accuracy=nan
2018-04-17 22:47:38,728 Epoch[183] Time cost=8.482
2018-04-17 22:47:38,890 Saved checkpoint to "./model/cars-finetuned-0184.params"
2018-04-17 22:47:43,541 Epoch[183] Validation-accuracy=0.679500
2018-04-17 22:47:45,897 Epoch[184] Batch [10]	Speed: 946.31 samples/sec	accuracy=0.998182
2018-04-17 22:47:48,011 Epoch[184] Batch [20]	Speed: 947.07 samples/sec	accuracy=0.998500
2018-04-17 22:47:50,106 Epoch[184] Batch [30]	Speed: 955.51 samples/sec	accuracy=0.998500
2018-04-17 22:47:52,193 Epoch[184] Batch [40]	Speed: 958.87 samples/sec	accuracy=0.999500
2018-04-17 22:47:52,193 Epoch[184] Train-accuracy=nan
2018-04-17 22:47:52,194 Epoch[184] Time cost=8.651
2018-04-17 22:4

2018-04-17 22:50:41,749 Epoch[197] Batch [20]	Speed: 947.77 samples/sec	accuracy=0.999500
2018-04-17 22:50:43,852 Epoch[197] Batch [30]	Speed: 951.23 samples/sec	accuracy=0.999000
2018-04-17 22:50:45,940 Epoch[197] Batch [40]	Speed: 958.54 samples/sec	accuracy=0.999000
2018-04-17 22:50:45,940 Epoch[197] Train-accuracy=nan
2018-04-17 22:50:45,941 Epoch[197] Time cost=8.493
2018-04-17 22:50:46,103 Saved checkpoint to "./model/cars-finetuned-0198.params"
2018-04-17 22:50:50,869 Epoch[197] Validation-accuracy=0.678375
2018-04-17 22:50:53,213 Epoch[198] Batch [10]	Speed: 951.06 samples/sec	accuracy=0.998182
2018-04-17 22:50:55,318 Epoch[198] Batch [20]	Speed: 950.50 samples/sec	accuracy=0.998500
2018-04-17 22:50:57,411 Epoch[198] Batch [30]	Speed: 955.78 samples/sec	accuracy=0.999500
2018-04-17 22:50:59,499 Epoch[198] Batch [40]	Speed: 958.50 samples/sec	accuracy=0.999000
2018-04-17 22:50:59,500 Epoch[198] Train-accuracy=nan
2018-04-17 22:50:59,500 Epoch[198] Time cost=8.629
2018-04-17 22:5

2018-04-17 22:53:49,615 Epoch[211] Batch [20]	Speed: 945.25 samples/sec	accuracy=0.999000
2018-04-17 22:53:51,718 Epoch[211] Batch [30]	Speed: 951.29 samples/sec	accuracy=1.000000
2018-04-17 22:53:53,808 Epoch[211] Batch [40]	Speed: 957.75 samples/sec	accuracy=0.999000
2018-04-17 22:53:53,810 Epoch[211] Train-accuracy=nan
2018-04-17 22:53:53,811 Epoch[211] Time cost=8.525
2018-04-17 22:53:53,977 Saved checkpoint to "./model/cars-finetuned-0212.params"
2018-04-17 22:53:58,548 Epoch[211] Validation-accuracy=0.679250
2018-04-17 22:54:00,895 Epoch[212] Batch [10]	Speed: 949.20 samples/sec	accuracy=0.999091
2018-04-17 22:54:03,000 Epoch[212] Batch [20]	Speed: 950.65 samples/sec	accuracy=1.000000
2018-04-17 22:54:05,092 Epoch[212] Batch [30]	Speed: 956.52 samples/sec	accuracy=0.999500
2018-04-17 22:54:07,179 Epoch[212] Batch [40]	Speed: 958.70 samples/sec	accuracy=0.999000
2018-04-17 22:54:07,181 Epoch[212] Train-accuracy=nan
2018-04-17 22:54:07,181 Epoch[212] Time cost=8.631
2018-04-17 22:5

2018-04-17 22:56:57,348 Epoch[225] Batch [20]	Speed: 947.01 samples/sec	accuracy=0.999000
2018-04-17 22:56:59,435 Epoch[225] Batch [30]	Speed: 958.75 samples/sec	accuracy=1.000000
2018-04-17 22:57:01,524 Epoch[225] Batch [40]	Speed: 958.00 samples/sec	accuracy=0.999000
2018-04-17 22:57:01,525 Epoch[225] Train-accuracy=nan
2018-04-17 22:57:01,525 Epoch[225] Time cost=8.641
2018-04-17 22:57:01,695 Saved checkpoint to "./model/cars-finetuned-0226.params"
2018-04-17 22:57:06,386 Epoch[225] Validation-accuracy=0.680875
2018-04-17 22:57:08,577 Epoch[226] Batch [10]	Speed: 1002.69 samples/sec	accuracy=0.998182
2018-04-17 22:57:10,698 Epoch[226] Batch [20]	Speed: 943.79 samples/sec	accuracy=0.999000
2018-04-17 22:57:12,793 Epoch[226] Batch [30]	Speed: 955.44 samples/sec	accuracy=0.999000
2018-04-17 22:57:14,877 Epoch[226] Batch [40]	Speed: 960.37 samples/sec	accuracy=0.999500
2018-04-17 22:57:14,877 Epoch[226] Train-accuracy=nan
2018-04-17 22:57:14,878 Epoch[226] Time cost=8.492
2018-04-17 22:

2018-04-17 23:00:03,342 Epoch[239] Batch [10]	Speed: 988.64 samples/sec	accuracy=0.999545
2018-04-17 23:00:05,456 Epoch[239] Batch [20]	Speed: 948.06 samples/sec	accuracy=0.998500
2018-04-17 23:00:07,549 Epoch[239] Batch [30]	Speed: 956.31 samples/sec	accuracy=0.999500
2018-04-17 23:00:09,425 Epoch[239] Train-accuracy=0.998889
2018-04-17 23:00:09,426 Epoch[239] Time cost=8.450
2018-04-17 23:00:09,600 Saved checkpoint to "./model/cars-finetuned-0240.params"
2018-04-17 23:00:14,716 Epoch[239] Validation-accuracy=0.680000


In [10]:
# Load the finetuned model for prediction on new images
import scipy.io # if this fails, install scipy 'conda install scipy'
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
import mxnet as mx
 
# define a simple data batch
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

num_epoch = 239

sym, arg_params, aux_params = mx.model.load_checkpoint('./model/cars-finetuned', num_epoch)
mod = mx.mod.Module(symbol=sym, context=mx.gpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)

# Load the label file for getting car name
label_file = scipy.io.loadmat('./data/cars_annos.mat')


def get_image(url, show=False):
    # download and show the image
    fname = mx.test_utils.download(url)
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    if img is None:
         return None
    if show:
         plt.imshow(img)
         plt.axis('off')
    # convert into format (batch, RGB, width, height)
    img = cv2.resize(img, (224, 224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return img

def predict(url,label_file):
    img = get_image(url, show=True)
    # compute the predict probabilities
    mod.forward(Batch([mx.nd.array(img)]))
    prob = mod.get_outputs()[0].asnumpy()
    # print the top-5
    prob = np.squeeze(prob)
    a = np.argsort(prob)[::-1]
    object_categories = [i[0] for i in label_file['class_names'][0]]
    for i in a[0:5]:
        print('probability=%f, class=%s' %(prob[i], object_categories[i]))

# Get prediction on an image

test_url = 'http://ai.stanford.edu/~jkrause/cars/car5.jpg'
#test_url = ''
predict(test_url,label_file)


MXNetError: [03:40:23] src/io/local_filesys.cc:166: Check failed: allow_null  LocalFileSystem: fail to open "./model/cars-finetuned-symbol.json"

Stack trace returned 10 entries:
[bt] (0) /home/ubuntu/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x276938) [0x7fca5bcee938]
[bt] (1) /home/ubuntu/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x28f59e0) [0x7fca5e36d9e0]
[bt] (2) /home/ubuntu/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x28ec10a) [0x7fca5e36410a]
[bt] (3) /home/ubuntu/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(MXSymbolCreateFromFile+0x4e) [0x7fca5de3bb7e]
[bt] (4) /home/ubuntu/anaconda3/envs/mxnet_p36/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call_unix64+0x4c) [0x7fcaec5deec0]
[bt] (5) /home/ubuntu/anaconda3/envs/mxnet_p36/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call+0x22d) [0x7fcaec5de87d]
[bt] (6) /home/ubuntu/anaconda3/envs/mxnet_p36/lib/python3.6/li